## Import Libraries

In [1]:
import os
import time
import copy


# 데이터 분석 라이브러리
import numpy as np
import pandas as pd

# 시각화 라이브러리
import matplotlib as mpl
import matplotlib.pyplot as plt

from PIL import Image

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
import timm

from __future__ import print_function, division
from torch.optim import lr_scheduler
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm

cudnn.benchmark = True
plt.ion()   # interactive mode

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
device

device(type='cuda', index=0)

## Bring csv file with path & label

In [4]:
traindata_path = pd.read_csv('train_data.csv')
valdata_path = pd.read_csv('val_data.csv')
testdata_path = pd.read_csv('test_data.csv')

In [5]:
print(len(traindata_path))

15126


## Hyper Parameter
Resize([224,224]) - https://www.programcreek.com/python/example/104834/torchvision.transforms.Resize

In [6]:
transform = transforms.Compose([
        transforms.Resize([384,384]),
#         transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
batch_size = 256
num_workers = 1

## Dataset

In [7]:
def readImage(path_data):
    path_to_image = []
    for path in path_data:
        img = Image.open(path)
        path_to_image.append(img)
    return path_to_image

In [8]:
class MyDataset(Dataset):
    def __init__(self, data, transform, train):
        self._repr_indent = 4
        self.data = data
        self.X = readImage(self.data['img_path'])
        self.y = self.data['label']
        self.transform = transform
        self.train = train
        self.classes = list(set(self.y))
    
    def __getitem__(self, idx):
        X, y = self.X[idx], None
#         X, y = self.X[idx], -1

        if self.transform:
            X = self.transform(X)
        if self.train:
            y = self.y[idx]

        return X,y
#         return torch.tensor(X), torch.tensor(y)
#         return X.clone().detach(), y.clone().detach()
    
    def __len__(self):
        return len(self.X)
    
    def __repr__(self):
        '''
        https://github.com/pytorch/vision/blob/master/torchvision/datasets/vision.py
        '''
        head = "My Custom Dataset : Mask Dataset"
        num_data = self._repr_indent*" " + "Number of datapoints: {}".format(self.__len__())
        num_classes = self._repr_indent*" " + "Number of classes: {}".format(len(self.classes))
        return '\n'.join([head, num_data, num_classes])

In [9]:
train_dataset = MyDataset(
    data = traindata_path,
    transform = transform,
    train = True
)
val_dataset = MyDataset(
    data = valdata_path,
    transform = transform,
    train = True
)
test_dataset = MyDataset(
    data = testdata_path,
    transform = transform,
    train = False
)

In [10]:
train_dataset

My Custom Dataset : Mask Dataset
    Number of datapoints: 15126
    Number of classes: 18

In [11]:
val_dataset

My Custom Dataset : Mask Dataset
    Number of datapoints: 3774
    Number of classes: 18

In [12]:
test_dataset

My Custom Dataset : Mask Dataset
    Number of datapoints: 12600
    Number of classes: 1

## DataLoader

In [13]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size = batch_size,
    num_workers = num_workers,
    shuffle = True,
    drop_last = False
)
val_loader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size = batch_size,
    num_workers = num_workers,
    shuffle = True,
    drop_last = False
)
test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size = batch_size,
    num_workers = num_workers,
    shuffle = True,
    drop_last = False
)

In [14]:
dataloaders = {
    'train': train_loader,
    'val': val_loader,
    'test': test_loader
}

In [15]:
dataloaders

{'train': <torch.utils.data.dataloader.DataLoader at 0x7f417502e100>,
 'val': <torch.utils.data.dataloader.DataLoader at 0x7f417502e160>,
 'test': <torch.utils.data.dataloader.DataLoader at 0x7f417502e280>}

## Model
model 생성 및 확인 - https://towardsdatascience.com/getting-started-with-pytorch-image-models-timm-a-practitioners-guide-4e77b4bf9055

In [16]:
timm.list_models('swin*',pretrained=True)

['swin_base_patch4_window7_224',
 'swin_base_patch4_window7_224_in22k',
 'swin_base_patch4_window12_384',
 'swin_base_patch4_window12_384_in22k',
 'swin_large_patch4_window7_224',
 'swin_large_patch4_window7_224_in22k',
 'swin_large_patch4_window12_384',
 'swin_large_patch4_window12_384_in22k',
 'swin_small_patch4_window7_224',
 'swin_tiny_patch4_window7_224']

In [17]:
model = timm.create_model('vit_small_patch32_384', pretrained=True, num_classes=18)

# num_ftrs = model.fc.in_features
# model.fc = nn.Linear(num_ftrs, 18)

model.get_classifier()

Linear(in_features=384, out_features=18, bias=True)

In [18]:
model(torch.randn(32,3,384,384)).shape

torch.Size([32, 18])

In [19]:
model = model.to(device)

loss_fn = torch.nn.CrossEntropyLoss() 

# Observe that all parameters are being optimized
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001) 

In [20]:
model = nn.Sequential(
    model,
    nn.Softmax(1)
)
model = model.to(device)
# model

In [21]:
# model.default_cfg

In [22]:
import torchsummary
# torchsummary.summary(model,(3,384,384))

## Train
transfer learning train model - https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html   
save checkpoint - https://tutorials.pytorch.kr/recipes/recipes/saving_and_loading_a_general_checkpoint.html   
f1 score - https://eunsukimme.github.io/ml/2019/10/21/Accuracy-Recall-Precision-F1-score/   
f1 = f1_score(labels, logits) - ypredict:0-17 ytarget:0-17   
https://www.delftstack.com/ko/howto/numpy/python-convert-tensor-to-numpy-array/   
tqdm - http://www.gisdeveloper.co.kr/?p=8636    


In [23]:
from sklearn.metrics import f1_score
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

can't convert cuda:0 device type tensor to numpy ---> 28 tmp = logits.numpy() -   
Can't call numpy() on Tensor that requires grad ---> 28 tmp = logits.cpu().numpy() - 

In [33]:
def train_model(model, loss_fn, optimizer, num_epochs=25):
    since = time.time()   
    best_acc = 0.0

    for epoch in range(num_epochs):
        # Each epoch has a training and validation phase
        for phase in ["train", "val"]:
            running_loss = 0.
            running_acc = 0.
            running_f1 = 0.
            running_f2 = 0.
            running_f3 = 0.
            if phase == "train":
                model.train() 
            elif phase == "val":
                model.eval() 

            for images, labels in tqdm(dataloaders[phase]):
                images = images.to(device)
                labels = labels.to(device)

                optimizer.zero_grad() 

                with torch.set_grad_enabled(phase == "train"):
                    logits = model(images)
                    _, preds = torch.max(logits, 1)
                    loss = loss_fn(logits, labels)
                    a = labels.detach().cpu().numpy()
                    b = preds.detach().cpu().numpy()
                    f1_1 = f1_score(a, b, average='micro')
                    f1_2 = f1_score(a, b, average='macro')
                    f1_3 = f1_score(a, b, average='weighted')

                    if phase == "train":
                        loss.backward() 
                        optimizer.step() 

                running_loss += loss.item() * images.size(0) 
                running_acc += torch.sum(preds == labels.data)
                running_f1 += f1_1 * images.size(0) 
                running_f2 += f1_2 * images.size(0) 
                running_f3 += f1_3 * images.size(0) 

            epoch_loss = running_loss / len(dataloaders[phase].dataset) 
            epoch_acc = running_acc / len(dataloaders[phase].dataset)
            epoch_f1 = running_f1 / (len(dataloaders[phase].dataset))
            epoch_f2 = running_f2 / (len(dataloaders[phase].dataset))
            epoch_f3 = running_f3 / (len(dataloaders[phase].dataset))

            print('Epoch {}/{} - {} Loss: {:.4f} Acc: {:.4f} '.format(epoch, num_epochs-1, phase, epoch_loss, epoch_acc))
            print('f1_score1: {:.4f} f1_score2: {:.4f} f1_score3: {:.4f}'.format(epoch_f1, epoch_f2, epoch_f3))
            
            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    return model

In [34]:
model = train_model(model, loss_fn, optimizer, num_epochs=5)


Epoch 0/4 - train Loss: 2.3971 Acc: 0.5858 
f1_score1: 0.5858 f1_score2: 0.1670 f1_score3: 0.4692



Epoch 0/4 - val Loss: 2.4261 Acc: 0.5556 
f1_score1: 0.5556 f1_score2: 0.1571 f1_score3: 0.4463




Epoch 1/4 - train Loss: 2.3894 Acc: 0.5928 
f1_score1: 0.5928 f1_score2: 0.1655 f1_score3: 0.4760



Epoch 1/4 - val Loss: 2.4568 Acc: 0.5238 
f1_score1: 0.5238 f1_score2: 0.1503 f1_score3: 0.4212




Epoch 2/4 - train Loss: 2.3860 Acc: 0.5958 
f1_score1: 0.5958 f1_score2: 0.1674 f1_score3: 0.4775



Epoch 2/4 - val Loss: 2.4274 Acc: 0.5543 
f1_score1: 0.5543 f1_score2: 0.1537 f1_score3: 0.4457




Epoch 3/4 - train Loss: 2.3743 Acc: 0.6084 
f1_score1: 0.6084 f1_score2: 0.1732 f1_score3: 0.4879



Epoch 3/4 - val Loss: 2.4061 Acc: 0.5771 
f1_score1: 0.5771 f1_score2: 0.1638 f1_score3: 0.4637




Epoch 4/4 - train Loss: 2.3673 Acc: 0.6140 
f1_score1: 0.6140 f1_score2: 0.1740 f1_score3: 0.4923



Epoch 4/4 - val Loss: 2.4270 Acc: 0.5556 
f1_score1: 0.5556 f1_score2: 0.1558 f1_score3: 0.4461

Training complete in 10m 38s
Best val Acc: 0.577107
